# Assignment 8: Dimensionality Reduction

Dataset(s) needed: MNIST ("Modified National Institute of Standards and Technology") dataset.

# Learning Objectives
- Be able to make application decisions regarding principal component analysis to train and test data 
- Produce a dimensionality reduction model.

In [27]:
#Load the MNIST dataset
import pandas as pd
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', as_frame=True)

display(mnist.data.head())
display(mnist.target.head())

pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel10  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0        0  ...         0         0         0         0         0         0   
1        0  ...         0         0         0         0         0         0   
2        0  ...         0         0         0         0         0         0   
3        0  ...         0         0         0         0         0         0   
4        0  ...         0         0         0         0         0         0   

   pixel781  pixel782  pixel783  pixel784  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  
3         0         0         0         0  
4         0         0         0         0  

[5 rows x 784 columns]

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

Question 1: Split the data into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing).

In [30]:
X_train = mnist.data.head(60000)
y_train = mnist.target.head(60000)

X_test = mnist.data.tail(10000)
y_test = mnist.target.tail(10000)

Question 2: Train a Logistic Regression classifier on the dataset. Set solver = lbfgs and random_state = 42. Hint you might also want to think about the multi_class parameter and if you need to set that.  You want to track how long the model takes to run.

In [31]:
from sklearn.linear_model import LogisticRegression
import time

all_vars_regression = LogisticRegression(
    max_iter=1000, random_state=42, solver="lbfgs", multi_class="multinomial"
)
all_vars_regression.fit(X_train, y_train)

# took 3m20s

/Users/matehal/Documents/mhaligowski/uw/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

Question 3: Evaluate the resulting model on the test set.

In [49]:
# Use score method to get accuracy of model
all_vars_regression.score(X_test, y_test)

0.9216

Question 4: Use PCA to reduce the dataset's dimensionality, with an explained variance ratio of 95%.

In [44]:
from sklearn.decomposition import PCA

PCA_model = PCA(n_components=0.95)
PCA_model.fit(mnist.data)

print(f"# of components {len(PCA_model.explained_variance_ratio_)}, variance {sum(PCA_model.explained_variance_ratio_)}")

# of components 154, variance 0.9503499702078614


Question 5: Train a new Logistic Regression classifier on the reduced dataset and see how long it takes. Use the same criteria as before just with PCA. Was training much faster? Explain your results

In [53]:
PCA_model.set_output(transform='pandas')
PCA_data = PCA_model.transform(mnist.data)
PCA_train = PCA_data.head(60000)
PCA_test = PCA_data.tail(10000)

pca_regression = LogisticRegression(
    max_iter=1000, random_state=42, solver="lbfgs", multi_class="multinomial"
)
pca_regression.fit(PCA_train, y_train)
# 1m24s

/Users/matehal/Documents/mhaligowski/uw/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

Question 6: Evaluate the new classifier on the test set: how does it compare to the previous classifier? Discuss the speed / accuracy trade-off and in which case you'd prefer a very slight drop in model performance for a x-time speedup in training.

In [55]:
# Use score method to get accuracy of model
pca_regression.score(PCA_test, y_test)

# PCA is faster and has a similar accuracy, even slightly better than the original. The factor of 
# training time is around 50%, so it's half of time! It might be a huge deal when dealing with
# very scarce resources, or when the difference is between training the model or not training at all.

0.9238

Question 7. Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

I had done PCA before, but the simple implementations to better understand the prionciples behind it. It took me some time to figure out what's going on with the MNIST dataset, but the documentation turned out to be pretty well-written. I made a slight mistake in the split part, but otherwise it went pretty smoothly, the PCA documentation on the sklearn webiste is very comprehensive.